<h2>Setup & Import</h2>

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Import data and setup dataframe
df = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')

In [4]:
# Import lookup tables
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
aircraft_types = pd.read_csv('data/L_AIRCRAFT_TYPE.csv')
city_markets = pd.read_csv('data/L_CITY_MARKET_ID.csv')

service_classes = pd.read_csv('data/L_SERVICE_CLASS.csv')
aircraft_groups = pd.read_csv('data/L_AIRCRAFT_GROUP.csv')
aircraft_congifs = pd.read_csv('data/L_AIRCRAFT_CONFIG.csv')

<h2>Data Cleansing</h2>

<h4>Carrier Classes</h4>

In [5]:
# Create simplified list of carrier class descriptions
carrier_classes_modified = []
for carrier_type in carrier_classes['Description']:
    index = carrier_type.index('Carrier')
    substr = carrier_type[:index-1]
    carrier_classes_modified.append(substr)

carrier_classes['Carrier Class'] = carrier_classes_modified

In [6]:
#Inspect updated carrier_classes
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [7]:
# Replace carrier groups with descriptions from modified lookup table
df = df.merge(carrier_classes, left_on='CARRIER_GROUP_NEW', right_on='Code', how='left')
df = df.drop(columns=['Code', 'Description', 'CARRIER_GROUP_NEW'])

<h4>World Area Codes (WAC)</h4>

In [8]:
# Replace WAC with descriptions from lookup table
df = df.merge(area_codes_table, left_on='ORIGIN_WAC', right_on='Code', how='left')
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

df = df.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
df['DEST_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

<h4>Aircraft type</h4>

In [13]:
aircraft_types.head()

,Code,Description
0,7,Aero Commander 200
1,8,Aero Macchi AL-60
2,9,Aeronca 7-AC
3,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A
4,20,Bellanca CH-300


In [10]:
df = df.merge(aircraft_types, left_on='AIRCRAFT_TYPE', right_on='Code', how='left')
df['AIRCRAFT_TYPE'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

<h4>City Markets</h4>

In [15]:
city_markets.head()

,Code,Description
0,30001,"Afognak Lake, AK"
1,30003,"Granite Mountain, AK"
2,30004,"Lik, AK"
3,30005,"Little Squaw, AK"
4,30006,"Kizhuyak, AK"


In [17]:
df = df.merge(city_markets, left_on='ORIGIN_CITY_MARKET_ID', right_on='Code', how='left')
df['ORIGIN_CITY_MARKET_ID'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

df = df.merge(city_markets, left_on='DEST_CITY_MARKET_ID', right_on='Code', how='left')
df['DEST_CITY_MARKET_ID'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

<h4>

<h2>Add Load Factor Metric</h2>

In [11]:
#Add Load Factor column on flight level
df['Load Factor'] = df['PASSENGERS'] / df['SEATS']
df['Load Factor'] = df['Load Factor'].fillna(0)

<h2>Display Final Table</h2>

In [18]:
df[:10]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Carrier Class,Load Factor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,"Block Island, RI",BID,"Block Island, RI",Rhode Island,United States,Rhode Island,"Westerly, RI",WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,Piper PA-32 (Cherokee 6),1,2024,1,1,1,F,DU,Commuter,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,"Block Island, RI",BID,"Block Island, RI",Rhode Island,United States,Rhode Island,"Westerly, RI",WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,Piper PA-32 (Cherokee 6),1,2024,3,7,1,F,DU,Commuter,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,"Westerly, RI",WST,"Westerly, RI",Rhode Island,United States,Rhode Island,"Block Island, RI",BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,Piper PA-32 (Cherokee 6),1,2024,1,1,1,F,DU,Commuter,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,"Westerly, RI",WST,"Westerly, RI",Rhode Island,United States,Rhode Island,"Block Island, RI",BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,Piper PA-32 (Cherokee 6),1,2024,3,7,1,F,DU,Commuter,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,"Martha's Vineyard, MA",MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,"Hyannis, MA",HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,Cessna C-402/402A/402B,1,2024,1,1,1,F,DU,Commuter,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,"Martha's Vineyard, MA",MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,"Nantucket, MA",ACK,"Nantucket, MA",Massachusetts,United States,Massachusetts,1,Cessna C-402/402A/402B,1,2024,1,2,1,F,DU,Commuter,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,"New York City, NY (Metropolitan Area)",HPN,"White Plains, NY",New York,United States,New York,"New York City, NY (Metropolitan Area)",JFK,"New York, NY",New York,United States,New York,6,Boeing 757-200,1,2024,1,2,1,L,DU,Major,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,"Boston, MA (Metropolitan Area)",BOS,"Boston, MA",Massachusetts,United States,Massachusetts,"Boston, MA (Metropolitan Area)",PVD,"Providence, RI",Rhode Island,United States,Rhode Island,6,Boeing 757-200,1,2024,3,7,1,L,DU,Major,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,"Christiansted, VI",STX,"Christiansted, VI",U.S. Virgin Islands,United States,U.S. Virgin Islands,"Vieques, PR",VQS,"Vieques, PR",Puerto Rico,United States,Puerto Rico,1,Cessna C-402/402A/402B,1,2024,2,4,1,F,DU,Commuter,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,"Milwaukee, WI",MKE,"Milwaukee, WI",Wisconsin,United States,Wisconsin,"Chicago, IL",MDW,"Chicago, IL",Illinois,United States,Illinois,6,Boeing 757-200,1,2024,2,4,1,L,DU,Major,0.0
